In [1]:
librarian::shelf(tidyverse, tidymodels, kableExtra, patchwork, 
                skimr, gridExtra, janitor, corrplot, scales,
                GGally, car, forcats, performance, glmmTMB, 
                splines, mgcv, DHARMa, zoo)

In [2]:
df <- read_csv("data/weatherAUS.csv")

Rows: 145460 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Location, WindGustDir, WindDir9am, WindDir3pm, RainToday, RainTom...
dbl  (16): MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed,...
date  (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
df %>% 
  head() %>% 
  kable()



|Date       |Location | MinTemp| MaxTemp| Rainfall| Evaporation| Sunshine|WindGustDir | WindGustSpeed|WindDir9am |WindDir3pm | WindSpeed9am| WindSpeed3pm| Humidity9am| Humidity3pm| Pressure9am| Pressure3pm| Cloud9am| Cloud3pm| Temp9am| Temp3pm|RainToday |RainTomorrow |
|:----------|:--------|-------:|-------:|--------:|-----------:|--------:|:-----------|-------------:|:----------|:----------|------------:|------------:|-----------:|-----------:|-----------:|-----------:|--------:|--------:|-------:|-------:|:---------|:------------|
|2008-12-01 |Albury   |    13.4|    22.9|      0.6|          NA|       NA|W           |            44|W          |WNW        |           20|           24|          71|          22|      1007.7|      1007.1|        8|       NA|    16.9|    21.8|No        |No           |
|2008-12-02 |Albury   |     7.4|    25.1|      0.0|          NA|       NA|WNW         |            44|NNW        |WSW        |            4|           22|          44|          25|      10

In [4]:
df %>% 
  tail() %>% 
  kable()



|Date       |Location | MinTemp| MaxTemp| Rainfall| Evaporation| Sunshine|WindGustDir | WindGustSpeed|WindDir9am |WindDir3pm | WindSpeed9am| WindSpeed3pm| Humidity9am| Humidity3pm| Pressure9am| Pressure3pm| Cloud9am| Cloud3pm| Temp9am| Temp3pm|RainToday |RainTomorrow |
|:----------|:--------|-------:|-------:|--------:|-----------:|--------:|:-----------|-------------:|:----------|:----------|------------:|------------:|-----------:|-----------:|-----------:|-----------:|--------:|--------:|-------:|-------:|:---------|:------------|
|2017-06-20 |Uluru    |     3.5|    21.8|        0|          NA|       NA|E           |            31|ESE        |E          |           15|           13|          59|          27|      1024.7|      1021.2|       NA|       NA|     9.4|    20.9|No        |No           |
|2017-06-21 |Uluru    |     2.8|    23.4|        0|          NA|       NA|E           |            31|SE         |ENE        |           13|           11|          51|          24|      10

In [5]:
df_clean <- df %>%
  clean_names() %>%
  mutate(
    date = as.Date(date),
    month = as.factor(month(date)),
    day = as.factor(wday(date, label = TRUE))
  ) %>%
  filter(!is.na(rainfall))

In [6]:
df_clean %>% 
  names()

 [1] "date"            "location"        "min_temp"        "max_temp"       
 [5] "rainfall"        "evaporation"     "sunshine"        "wind_gust_dir"  
 [9] "wind_gust_speed" "wind_dir9am"     "wind_dir3pm"     "wind_speed9am"  
[13] "wind_speed3pm"   "humidity9am"     "humidity3pm"     "pressure9am"    
[17] "pressure3pm"     "cloud9am"        "cloud3pm"        "temp9am"        
[21] "temp3pm"         "rain_today"      "rain_tomorrow"   "month"          
[25] "day"            

In [7]:
# Check for duplicates

duplicates <- df_clean %>% 
  get_dupes()

print(paste("Number of duplicate rows: ", nrow(duplicates)))

No variable names specified - using all columns.

No duplicate combinations found of: date, location, min_temp, max_temp, rainfall, evaporation, sunshine, wind_gust_dir, wind_gust_speed, ... and 16 other variables
[1] "Number of duplicate rows:  0"


In [8]:
# Day Distribution

day_tab <- df_clean %>% 
  filter(rainfall > 0) %>% 
  tabyl(day) %>% 
  adorn_pct_formatting() %>% 
  arrange(desc(n))

print(day_tab)

 day    n percent
 Tue 7508   14.7%
 Mon 7480   14.6%
 Fri 7378   14.4%
 Wed 7342   14.4%
 Thu 7314   14.3%
 Sat 7057   13.8%
 Sun 7040   13.8%


In [9]:
# Month distribution

month_tab <- df_clean %>% 
  filter(rainfall > 0) %>% 
  tabyl(month) %>% 
  adorn_pct_formatting() %>% 
  arrange(desc(n))

print(month_tab)

 month    n percent
     6 5448   10.7%
     7 5250   10.3%
     5 4937    9.7%
     8 4704    9.2%
     3 4444    8.7%
     9 4234    8.3%
     4 4001    7.8%
    10 3770    7.4%
    11 3760    7.4%
     1 3702    7.2%
    12 3562    7.0%
     2 3307    6.5%


In [10]:
cat("\nCross-tabulation: Month vs Day:\n")
cross_tab <- df_clean %>% 
  filter(rainfall > 0) %>% 
  tabyl(month, day) %>% 
  adorn_totals(c("row", "col"))
print(cross_tab)


Cross-tabulation: Month vs Day:
 month  Sun  Mon  Tue  Wed  Thu  Fri  Sat Total
     1  536  570  514  482  493  567  540  3702
     2  480  530  469  466  443  471  448  3307
     3  636  645  639  592  662  644  626  4444
     4  578  597  566  604  579  515  562  4001
     5  710  722  765  714  678  681  667  4937
     6  766  801  804  797  760  753  767  5448
     7  694  717  728  796  773  818  724  5250
     8  612  702  694  679  689  687  641  4704
     9  516  571  616  639  648  659  585  4234
    10  507  606  579  550  517  456  555  3770
    11  556  526  554  480  538  575  531  3760
    12  449  493  580  543  534  552  411  3562
 Total 7040 7480 7508 7342 7314 7378 7057 51119


In [11]:
print(paste("Total Missing values: ", sum(is.na(df))))

[1] "Total Missing values:  343248"


In [12]:
missing_tab <- df_clean %>%
  summarise(across(everything(), ~ mean(is.na(.)) * 100)) %>%
  pivot_longer(everything(), names_to = "column", values_to = "pct_missing") %>%
  arrange(desc(pct_missing))

missing_tab %>% 
  kable()



|column          | pct_missing|
|:---------------|-----------:|
|sunshine        |  47.6937250|
|evaporation     |  42.5375706|
|cloud3pm        |  39.9960619|
|cloud9am        |  37.5044832|
|pressure3pm     |   9.8404349|
|pressure9am     |   9.8031632|
|wind_dir9am     |   6.8840147|
|wind_gust_dir   |   6.8390073|
|wind_gust_speed |   6.7968129|
|wind_dir3pm     |   2.6716081|
|humidity3pm     |   2.5527606|
|temp3pm         |   1.9310966|
|wind_speed3pm   |   1.8614758|
|humidity9am     |   1.0928347|
|rain_tomorrow   |   0.9929746|
|wind_speed9am   |   0.7672347|
|temp9am         |   0.4817193|
|min_temp        |   0.3424778|
|max_temp        |   0.3305227|
|date            |   0.0000000|
|location        |   0.0000000|
|rainfall        |   0.0000000|
|rain_today      |   0.0000000|
|month           |   0.0000000|
|day             |   0.0000000|

In [13]:
# Obtain the summary statistics for the target variable

rainfall_stats <- df_clean %>% 
  summarise(
    n = n(),
    mean = mean(rainfall),
    median = median(rainfall),
    sd = sd(rainfall),
    min = min(rainfall),
    max = max(rainfall),
    q25 = quantile(rainfall, 0.25),
    q75 = quantile(rainfall, .75),
    iqr = IQR(rainfall),
    n_zeros = sum(rainfall == 0),
    pct_zeros = mean(rainfall == 0) * 100,
    n_large = sum(rainfall > 100),
    pct_large = mean(rainfall > 100) * 100,
    skewness = moments::skewness(rainfall),
    kurtosis = moments::kurtosis(rainfall)
  )

t(rainfall_stats) %>% 
  kable()



|          |             |
|:---------|------------:|
|n         | 1.421990e+05|
|mean      | 2.360918e+00|
|median    | 0.000000e+00|
|sd        | 8.478060e+00|
|min       | 0.000000e+00|
|max       | 3.710000e+02|
|q25       | 0.000000e+00|
|q75       | 8.000000e-01|
|iqr       | 8.000000e-01|
|n_zeros   | 9.108000e+04|
|pct_zeros | 6.405108e+01|
|n_large   | 1.510000e+02|
|pct_large | 1.061892e-01|
|skewness  | 9.836122e+00|
|kurtosis  | 1.811458e+02|

In [14]:

rain_check <- df_clean %>%
  summarise(
    total_days = n(),
    dry_days = sum(rainfall == 0),
    rainy_days = sum(rainfall > 0),
    zero_inflation_pct = (dry_days / total_days) * 100
  )

rain_check %>% 
  kable()



| total_days| dry_days| rainy_days| zero_inflation_pct|
|----------:|--------:|----------:|------------------:|
|     142199|    91080|      51119|           64.05108|

In [15]:
numeric_cols <- df_clean %>% 
select(where(is.numeric)) %>% 
names()

numeric_cols <- numeric_cols[numeric_cols != "rainfall"]


cors <- df_clean%>%
rstatix::cor_test(vars = "rainfall", vars2 = numeric_cols, method = "spearman") %>%
filter(!is.na(cor)) %>%
arrange(desc(abs(cor))) %>%
dplyr::select(var2, cor, p) %>%
mutate(interpretation = case_when(
  abs(cor) < 0.1 ~ "Negligible",
  abs(cor) < 0.3 ~ "Small",
  abs(cor) < 0.5 ~ "Moderate",
  TRUE ~ "Large"
))

cors

Warning message:
Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(numeric_cols)

  # Now:
  data %>% select(all_of(numeric_cols))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This warning is displayed once per session.
Call ]8;;x-r-run:lifecycle::last_lifecycle_warnings()lifecycle::last_lifecycle_warnings()]8;; to see where this warning was
generated. 


# A tibble: 15 × 4
   var2               cor         p interpretation
   <chr>            <dbl>     <dbl> <chr>         
 1 humidity9am      0.44  0         Moderate      
 2 humidity3pm      0.44  0         Moderate      
 3 sunshine        -0.4   0         Moderate      
 4 cloud9am         0.37  0         Moderate      
 5 cloud3pm         0.32  0         Moderate      
 6 evaporation     -0.31  0         Moderate      
 7 temp3pm         -0.31  0         Moderate      
 8 max_temp        -0.3   0         Moderate      
 9 pressure9am     -0.15  0         Small         
10 temp9am         -0.15  0         Small         
11 wind_gust_speed  0.13  0         Small         
12 wind_speed9am    0.083 1.02e-212 Negligible    
13 wind_speed3pm    0.068 5.65e-143 Negligible    
14 pressure3pm     -0.063 1.64e-114 Negligible    
15 min_temp         0.022 3.04e- 17 Negligible    

In [16]:
print(paste("Total Missing values: ", sum(is.na(df_clean))))

[1] "Total Missing values:  314146"


In [18]:
missing_val <- function(df){
  missing_tab <- df %>% 
    summarise(across(everything(), ~mean(is.na(.)) * 100)) %>% 
    pivot_longer(everything(), names_to = "column", values_to = "pct_missing") %>% 
    arrange(desc(pct_missing))

  return(missing_tab %>% kable())
}


missing_val(df_clean)



|column          | pct_missing|
|:---------------|-----------:|
|sunshine        |  47.6937250|
|evaporation     |  42.5375706|
|cloud3pm        |  39.9960619|
|cloud9am        |  37.5044832|
|pressure3pm     |   9.8404349|
|pressure9am     |   9.8031632|
|wind_dir9am     |   6.8840147|
|wind_gust_dir   |   6.8390073|
|wind_gust_speed |   6.7968129|
|wind_dir3pm     |   2.6716081|
|humidity3pm     |   2.5527606|
|temp3pm         |   1.9310966|
|wind_speed3pm   |   1.8614758|
|humidity9am     |   1.0928347|
|rain_tomorrow   |   0.9929746|
|wind_speed9am   |   0.7672347|
|temp9am         |   0.4817193|
|min_temp        |   0.3424778|
|max_temp        |   0.3305227|
|date            |   0.0000000|
|location        |   0.0000000|
|rainfall        |   0.0000000|
|rain_today      |   0.0000000|
|month           |   0.0000000|
|day             |   0.0000000|

In [19]:
df_sorted <- df_clean %>% 
  arrange(location, date) 

In [20]:
df_flagged <- df_sorted %>% 
  mutate(
    sunshine_imp_flagged = ifelse(is.na(sunshine), 1, 0),
    evap_imp_flagged = ifelse(is.na(evaporation), 1, 0),
    cloud3pm_imp_flagged = ifelse(is.na(cloud3pm), 1, 0),
    cloud9am_imp_flagged = ifelse(is.na(cloud9am), 1, 0),
  )



In [21]:
# Interpolation for small continuous variables
# Logic: if the weather on monday is 20 degrees, on Wednesday its 22, then on tuesday it might be 21

interp_vars <- c("min_temp", "max_temp", "temp9am", "temp3pm",
                   "humidity9am", "humidity3pm", "pressure9am", "pressure3pm",
                   "wind_speed9am", "wind_speed3pm", "wind_gust_speed")

# The idea is that we do not gues if we are having more than 3 days missing,
#  while keeping the leading/trailing NAs s they dont crash on empty groups
df_interp <- df_flagged %>% 
  group_by(location) %>% 
  mutate(across(all_of(interp_vars),
               ~na.approx(., maxgap = 3, na.rm = FALSE, rule = 2))) %>% 
  ungroup()

df_interp

# A tibble: 142,199 × 29
   date       location min_temp max_temp rainfall evaporation sunshine
   <date>     <chr>       <dbl>    <dbl>    <dbl>       <dbl>    <dbl>
 1 2008-07-01 Adelaide      8.8     15.7      5           1.6      2.6
 2 2008-07-02 Adelaide     12.7     15.8      0.8         1.4      7.8
 3 2008-07-03 Adelaide      6.2     15.1      0           1.8      2.1
 4 2008-07-04 Adelaide      5.3     15.9      0           1.4      8  
 5 2008-07-05 Adelaide      9.8     15.4      0          NA        0.9
 6 2008-07-07 Adelaide      7.6     11.2     16.2         4.6      1.1
 7 2008-07-08 Adelaide      5.3     13.5     17           0.6      2.1
 8 2008-07-09 Adelaide      8.4     14.3      1.8         1.6      0.8
 9 2008-07-10 Adelaide      9.5     13.1      9           1.2      7.2
10 2008-07-11 Adelaide      8.2     14        0.2         2.8      4.3
# ℹ 142,189 more rows
# ℹ 22 more variables: wind_gust_dir <chr>, wind_gust_speed <dbl>,
#   wind_dir9am <chr>, wind_dir3pm

In [22]:
# For the columns with the highest missing data interpolation will fail
# Logic: Since interpolation fails, we will use the typical value fo this month and location

fill_hierarchical <- function(x, loc, mth){
  # If value exists, keep it
  # If missing try location-month median
  # if thats missing (loc has no data), try globa-month median,
  # Else just fallback to global median

  df_temp <- data.frame(val = x, loc = loc, mth = mth)

  df_temp <- df_temp %>% 
    group_by(loc, mth) %>% 
    mutate(med_loc_mth = median(val, na.rm = TRUE)) %>% 
    ungroup() %>% 
    group_by(mth) %>% 
    mutate(med_mth = median(val, na.rm = TRUE)) %>% 
    ungroup()

  # Take the first non NA value
  coalesce(df_temp$val, df_temp$med_loc_mth, df_temp$med_mth, median(x, na.rm=TRUE))
  
}

df_filled <- df_interp %>%
    mutate(
      sunshine    = fill_hierarchical(sunshine, location, month),
      evaporation = fill_hierarchical(evaporation, location, month),
      cloud3pm    = fill_hierarchical(cloud3pm, location, month),
      cloud9am    = fill_hierarchical(cloud9am, location, month)
    )

In [23]:
# Categorical variables (Wind Direction)
# Logic: "Last Observation Carried Forward" (LOCF)
  
df_final <- df_filled %>%
  group_by(location) %>%
  fill(c(wind_dir9am, wind_dir3pm, wind_gust_dir), .direction = "downup") %>%
  ungroup() %>%
  filter(!is.na(rain_tomorrow))

In [ ]:
calc_mode <- function(x) {
  ux <- unique(na.omit(x))
  if(length(ux) == 0) return(NA)
  ux[which.max(tabulate(match(x, ux)))]
}

final_cleanup <- function(df) {
  
  df %>%
    group_by(location, month) %>%
    mutate(
      # Fill with median of this Location+Month
      across(c(pressure9am, pressure3pm, wind_gust_speed, 
               humidity9am, humidity3pm, temp9am, temp3pm, 
               min_temp, max_temp, wind_speed9am, wind_speed3pm),
             ~ifelse(is.na(.), median(., na.rm = TRUE), .)),
      
      # Fill with mode of this Location+Month
      wind_gust_dir = ifelse(is.na(wind_gust_dir), calc_mode(wind_gust_dir), wind_gust_dir)
    ) %>%
    ungroup() %>%
    
    # 3. Regional Fallback
    # If a location is 100% missing for a variable, use the MONTHLY median across ALL locations
    group_by(month) %>%
    mutate(
      across(c(pressure9am, pressure3pm, wind_gust_speed, 
               humidity9am, humidity3pm, temp9am, temp3pm, 
               min_temp, max_temp, wind_speed9am, wind_speed3pm),
             ~ifelse(is.na(.), median(., na.rm = TRUE), .)),
      
      # Fallback for wind direction
      wind_gust_dir = ifelse(is.na(wind_gust_dir), calc_mode(wind_gust_dir), wind_gust_dir)
    ) %>%
    ungroup()
}

df_tidy <- final_cleanup(df_final)

In [30]:
missing_val(df_tidy)



|column               | pct_missing|
|:--------------------|-----------:|
|date                 |           0|
|location             |           0|
|min_temp             |           0|
|max_temp             |           0|
|rainfall             |           0|
|evaporation          |           0|
|sunshine             |           0|
|wind_gust_dir        |           0|
|wind_gust_speed      |           0|
|wind_dir9am          |           0|
|wind_dir3pm          |           0|
|wind_speed9am        |           0|
|wind_speed3pm        |           0|
|humidity9am          |           0|
|humidity3pm          |           0|
|pressure9am          |           0|
|pressure3pm          |           0|
|cloud9am             |           0|
|cloud3pm             |           0|
|temp9am              |           0|
|temp3pm              |           0|
|rain_today           |           0|
|rain_tomorrow        |           0|
|month                |           0|
|day                  |           0|

# Trim the columns


In [39]:

vif_check <- lm(rainfall ~ ., data = df_clean)
check_collinearity(vif_check)

Model matrix is rank deficient. VIFs may not be sensible.


# Check for Multicollinearity

Low Correlation

              Term  VIF           VIF 95% CI adj. VIF Tolerance
              date 1.24 [    1.24,     1.25]     1.11      0.80
       evaporation 2.38 [    2.36,     2.40]     1.54      0.42
   wind_gust_speed 2.85 [    2.82,     2.87]     1.69      0.35
     wind_speed9am 2.16 [    2.15,     2.18]     1.47      0.46
     wind_speed3pm 2.32 [    2.30,     2.34]     1.52      0.43
       humidity9am 4.63 [    4.58,     4.67]     2.15      0.22
          cloud3pm 4.04 [    4.00,     4.08]     2.01      0.25
        rain_today 1.45 [    1.44,     1.46]     1.20      0.69
     rain_tomorrow 1.54 [    1.53,     1.55]     1.24      0.65
               day 1.03 [    1.02,     1.03]     1.00      0.97
 cloud3pm_imp_flag 4.08 [    4.04,     4.12]     2.02      0.25
 cloud_development 3.31 [    3.28,     3.33]     1.82      0.30
 Tolerance 95% CI
     [0.80, 0.81]
     [0.42, 0.42]
     [0.35, 0.35]
     [0.46, 0.47]
     [0.43, 0.43]
     [0.21, 

In [ ]:
df_clean <- df_clean %>% 
  select(-c(min_temp, temp9am, temp3pm, pressure9am, wind_dir3pm,
            wind_dir9am, location, date, rain_tomorrow,
            sunshine, evaporation, cloud3pm, cloud9am))

In [22]:

m1_zigamma <- glmmTMB(
  rainfall ~ max_temp + wind_gust_speed + humidity3pm + pressure3pm + rain_today + month,
  ziformula = ~ humidity3pm + pressure3pm + month, 
  data = df_clean,
  family = ziGamma(link = "log")
)

summary(m1_zigamma)

 Family: Gamma  ( log )
Formula:          
rainfall ~ max_temp + wind_gust_speed + humidity3pm + pressure3pm +  
    rain_today + month
Zero inflation:            ~humidity3pm + pressure3pm + month
Data: df_clean

      AIC       BIC    logLik -2*log(L)  df.resid 
 308357.1  308667.5 -154146.5  308293.1    120851 


Dispersion estimate for Gamma family (sigma^2): 0.707 

Conditional model:
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)      8.8120567  0.6621234    13.3  < 2e-16 ***
max_temp         0.0192311  0.0010079    19.1  < 2e-16 ***
wind_gust_speed  0.0084532  0.0003088    27.4  < 2e-16 ***
humidity3pm      0.0128827  0.0002545    50.6  < 2e-16 ***
pressure3pm     -0.0108645  0.0006401   -17.0  < 2e-16 ***
rain_todayYes    2.8902092  0.0091246   316.8  < 2e-16 ***
month2           0.0700942  0.0221127     3.2  0.00153 ** 
month3           0.0023127  0.0205786     0.1  0.91052    
month4          -0.0523097  0.0217401    -2.4  0.01612 *  
month5          -

In [ ]:
# Generate and visualize predictions

In [ ]:
# use tweedie glmmTMB instead of the hurdle zigamma


In [ ]:
# Use GAM with tweedie to capture non linear relationships

In [ ]:
# Statistical model comparison (AIC x BIC) 

In [ ]:
# Diagnose the GAM using Dharma

In [ ]:
# Bayesian Inference?